In [139]:
#导入相关包
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
#保值率计算函数
def rate_residual(df):
    df['rate_residual'] = df['price']/df['price_new']
    return(df)

In [5]:
#新车价格对数化函数
def price_log(df):
    df['price_log'] = df['price_new'].apply(np.log1p)
    return(df)

In [ ]:
#车龄计算函数
def num_day(df):
    now=datetime(2022,11,11)
    df['date_regi']=df['date_regi'].apply(lambda x: datetime.strptime(x, '%Y-%m'))
    df['num_day']=df['date_regi'].apply(lambda x: now-x)
    df['num_day']=df['num_day'].apply(lambda x: x.days)
    return(df)

In [145]:
#里程车龄偏差率计算函数
##从多元线性回归模型可以看出车龄对预测保值率的贡献远大于里程，猜想是因为里程表读数容易被篡改
##自然可以联想到，车龄和里程数有较强的正相关关系，如果某一辆车这两者相差过大，我们怀疑有造假的可能，其价格可能会偏低
###方法一：构造简单线性回归，用车龄预测里程数，看偏差比率
def mile_diff(df):
    model = LinearRegression()

    x = df[['num_day']]
    y = df[['mileage']]

    model.fit(x,y)

    df['predict_mileage'] = model.predict(df[['num_day']])
    df['mile_diff'] = df['predict_mileage'] - df['mileage'] 
    df['mile_diff'] = 2*df['mile_diff'] / (df['predict_mileage'] + df['mileage'])
    return(df)
    
###方法二：用里程数除以车龄，得到每天行驶里程
def mileperday(df):
    df['mileperday'] = df['mileage'] / df['num_day']
    return(df)

In [143]:
#品牌占比计算函数，代表该车的市场保有量
def brand_ratio(df):
    df['brand_ratio'] = df['brand'].apply(lambda x: df['brand'].value_counts()[x]/32018)
    return(df)

In [144]:
#标题长度计算函数
def len_title(df):
    df['len_title'] = df['title'].apply(lambda x: len(x))